In [11]:
import numpy as np
import pandas as pd
from scipy import linalg
from math import sqrt
from tabulate import tabulate
import math

In [12]:
np.set_printoptions(formatter={'float_kind':"{:.8f}".format})
pd.options.display.float_format = '{:.8f}'.format

# Data creation

In [13]:
def pretty_view(a):
    pret=pd.DataFrame(data=a, index=list(range(1, a.shape[0]+1)), columns=list(range(1, a.shape[1]+1)))
    display(pret)

Hilbert matrices of order 4, 5

In [14]:
h4=linalg.hilbert(5)
pretty_view(h4)

,1,2,3,4,5
1,1.00000000,0.50000000,0.33333333,0.25000000,0.20000000
2,0.50000000,0.33333333,0.25000000,0.20000000,0.16666667
3,0.33333333,0.25000000,0.20000000,0.16666667,0.14285714
4,0.25000000,0.20000000,0.16666667,0.14285714,0.12500000
5,0.20000000,0.16666667,0.14285714,0.12500000,0.11111111


In [15]:
h6=linalg.hilbert(6)
pretty_view(h6)

,1,2,3,4,5,6
1,1.00000000,0.50000000,0.33333333,0.25000000,0.20000000,0.16666667
2,0.50000000,0.33333333,0.25000000,0.20000000,0.16666667,0.14285714
3,0.33333333,0.25000000,0.20000000,0.16666667,0.14285714,0.12500000
4,0.25000000,0.20000000,0.16666667,0.14285714,0.12500000,0.11111111
5,0.20000000,0.16666667,0.14285714,0.12500000,0.11111111,0.10000000
6,0.16666667,0.14285714,0.12500000,0.11111111,0.10000000,0.09090909


In [16]:
h10=linalg.hilbert(10)
pretty_view(h10)

,1,2,3,4,5,6,7,8,9,10
1,1.00000000,0.50000000,0.33333333,0.25000000,0.20000000,0.16666667,0.14285714,0.12500000,0.11111111,0.10000000
2,0.50000000,0.33333333,0.25000000,0.20000000,0.16666667,0.14285714,0.12500000,0.11111111,0.10000000,0.09090909
3,0.33333333,0.25000000,0.20000000,0.16666667,0.14285714,0.12500000,0.11111111,0.10000000,0.09090909,0.08333333
4,0.25000000,0.20000000,0.16666667,0.14285714,0.12500000,0.11111111,0.10000000,0.09090909,0.08333333,0.07692308
5,0.20000000,0.16666667,0.14285714,0.12500000,0.11111111,0.10000000,0.09090909,0.08333333,0.07692308,0.07142857
6,0.16666667,0.14285714,0.12500000,0.11111111,0.10000000,0.09090909,0.08333333,0.07692308,0.07142857,0.06666667
7,0.14285714,0.12500000,0.11111111,0.10000000,0.09090909,0.08333333,0.07692308,0.07142857,0.06666667,0.06250000
8,0.12500000,0.11111111,0.10000000,0.09090909,0.08333333,0.07692308,0.07142857,0.06666667,0.06250000,0.05882353
9,0.11111111,0.10000000,0.09090909,0.08333333,0.07692308,0.07142857,0.06666667,0.06250000,0.05882353,0.05555556
10,0.10000000,0.09090909,0.08333333,0.07692308,0.07142857,0.06666667,0.06250000,0.05882353,0.05555556,0.05263158


Diagonally dominant matrix

# Methods realisation

In [17]:
def max_abs(A):
    max_elem = 0
    i_max = 0
    j_max = 0
    for i in range(A.shape[0]):
        for j in range(i + 1, A.shape[0]):
            if abs(max_elem) < abs(A[i, j]):
                max_elem = A[i, j]
                i_max = i
                j_max = j
    return i_max, j_max

In [18]:
def max_gersh(A, n_iter, r, i, j):
    max_elem = 0
    max_i = 0
    max_j = 0
    if n_iter == 0:
        for i in range(A.shape[0]):
            for j in range(A.shape[0]):
                if i != j:
                    r[i] += abs(A[i, j]) ** 2
    else:
        for k in (i, j):
            r[k] = 0
            for j in range(A.shape[0]):
                if k != j:
                    r[k] += abs(A[k, j]) ** 2
    for i in range(A.shape[0]):
        if max_elem < r[i]:
            max_elem = r[i]
            max_i = i
    max_elem = 0
    for j in range(A.shape[0]):
        if max_elem < abs(A[max_i, j]) and max_i != j:
            max_elem = A[max_i, j]
            max_j = j
    return max_i, max_j


In [19]:
def jacobi_method(A, eps, gersh=True):
    n_iter = 0
    r = np.array(np.zeros(A.shape[0]))
    max_i = 0
    max_j = 0
    while True:
        H = np.eye(A.shape[0], dtype=float)
        if gersh:
            max_i, max_j = max_gersh(A, n_iter, r, max_i, max_j)  # через круги Гершгорина
        else:
            max_i, max_j = max_abs(A)  # через наибольший наддиагональный по модулю
        if abs(A[max_i, max_j]) < eps:
            return np.diag(A), n_iter
        n_iter += 1
        phi = 1 / 2 * (math.atan((2 * A[max_i, max_j]) / (A[max_i, max_i] - A[max_j, max_j])))
        H[max_i, max_i] = math.cos(phi)
        H[max_j, max_j] = math.cos(phi)
        H[max_i, max_j] = - math.sin(phi)
        H[max_j, max_i] = math.sin(phi)
        A = H.T @ A @ H

# Print results

In [20]:
def print_result(A):
    pretty_view(A)
    np_eig = np.linalg.eig(A)[0]
    table=[['Приближение', "n_iter Гершорина","n_iter max наддиагональный", 
                'np_eig - gers_eig', 'np_eig - maxabs_eig']]
    for eps in (1e-2, 1e-3, 1e-4, 1e-5):
        row=[]
        row.append(eps)
        gersh_eig, n_iter_1 = jacobi_method(A, eps)
        maxabs_eig, n_iter_2 = jacobi_method(A, eps, False)
        row.append(n_iter_1)
        row.append(n_iter_2)
        row.append(np.linalg.norm(np_eig - gersh_eig))
        row.append(np.linalg.norm(np_eig - maxabs_eig))
        table.append(row)
    return print(tabulate(table, tablefmt='fancy_grid', numalign="right"))

In [21]:
list_of_matrixes=[h4, h6, h10]

In [22]:
for matrix in list_of_matrixes:
    print_result(matrix)

,1,2,3,4,5
1,1.00000000,0.50000000,0.33333333,0.25000000,0.20000000
2,0.50000000,0.33333333,0.25000000,0.20000000,0.16666667
3,0.33333333,0.25000000,0.20000000,0.16666667,0.14285714
4,0.25000000,0.20000000,0.16666667,0.14285714,0.12500000
5,0.20000000,0.16666667,0.14285714,0.12500000,0.11111111


╒═════════════╤══════════════════╤════════════════════════════╤══════════════════════╤══════════════════════╕
│ Приближение │ n_iter Гершорина │ n_iter max наддиагональный │ np_eig - gers_eig    │ np_eig - maxabs_eig  │
├─────────────┼──────────────────┼────────────────────────────┼──────────────────────┼──────────────────────┤
│ 0.01        │ 9                │ 11                         │ 0.010716877008945656 │ 0.01071954538455553  │
├─────────────┼──────────────────┼────────────────────────────┼──────────────────────┼──────────────────────┤
│ 0.001       │ 9                │ 17                         │ 0.010716877008945656 │ 0.0159667609888987   │
├─────────────┼──────────────────┼────────────────────────────┼──────────────────────┼──────────────────────┤
│ 0.0001      │ 9                │ 18                         │ 0.010716877008945656 │ 0.015918121692733208 │
├─────────────┼──────────────────┼────────────────────────────┼──────────────────────┼──────────────────────┤
│ 1e-05   

,1,2,3,4,5,6
1,1.00000000,0.50000000,0.33333333,0.25000000,0.20000000,0.16666667
2,0.50000000,0.33333333,0.25000000,0.20000000,0.16666667,0.14285714
3,0.33333333,0.25000000,0.20000000,0.16666667,0.14285714,0.12500000
4,0.25000000,0.20000000,0.16666667,0.14285714,0.12500000,0.11111111
5,0.20000000,0.16666667,0.14285714,0.12500000,0.11111111,0.10000000
6,0.16666667,0.14285714,0.12500000,0.11111111,0.10000000,0.09090909


╒═════════════╤══════════════════╤════════════════════════════╤═════════════════════╤══════════════════════╕
│ Приближение │ n_iter Гершорина │ n_iter max наддиагональный │ np_eig - gers_eig   │ np_eig - maxabs_eig  │
├─────────────┼──────────────────┼────────────────────────────┼─────────────────────┼──────────────────────┤
│ 0.01        │ 11               │ 14                         │ 0.01588876515479488 │ 0.015832569910341814 │
├─────────────┼──────────────────┼────────────────────────────┼─────────────────────┼──────────────────────┤
│ 0.001       │ 11               │ 22                         │ 0.01588876515479488 │ 0.02281258317850855  │
├─────────────┼──────────────────┼────────────────────────────┼─────────────────────┼──────────────────────┤
│ 0.0001      │ 11               │ 28                         │ 0.01588876515479488 │ 0.02265837042984893  │
├─────────────┼──────────────────┼────────────────────────────┼─────────────────────┼──────────────────────┤
│ 1e-05       │ 11 

,1,2,3,4,5,6,7,8,9,10
1,1.00000000,0.50000000,0.33333333,0.25000000,0.20000000,0.16666667,0.14285714,0.12500000,0.11111111,0.10000000
2,0.50000000,0.33333333,0.25000000,0.20000000,0.16666667,0.14285714,0.12500000,0.11111111,0.10000000,0.09090909
3,0.33333333,0.25000000,0.20000000,0.16666667,0.14285714,0.12500000,0.11111111,0.10000000,0.09090909,0.08333333
4,0.25000000,0.20000000,0.16666667,0.14285714,0.12500000,0.11111111,0.10000000,0.09090909,0.08333333,0.07692308
5,0.20000000,0.16666667,0.14285714,0.12500000,0.11111111,0.10000000,0.09090909,0.08333333,0.07692308,0.07142857
6,0.16666667,0.14285714,0.12500000,0.11111111,0.10000000,0.09090909,0.08333333,0.07692308,0.07142857,0.06666667
7,0.14285714,0.12500000,0.11111111,0.10000000,0.09090909,0.08333333,0.07692308,0.07142857,0.06666667,0.06250000
8,0.12500000,0.11111111,0.10000000,0.09090909,0.08333333,0.07692308,0.07142857,0.06666667,0.06250000,0.05882353
9,0.11111111,0.10000000,0.09090909,0.08333333,0.07692308,0.07142857,0.06666667,0.06250000,0.05882353,0.05555556
10,0.10000000,0.09090909,0.08333333,0.07692308,0.07142857,0.06666667,0.06250000,0.05882353,0.05555556,0.05263158


╒═════════════╤══════════════════╤════════════════════════════╤═════════════════════╤══════════════════════╕
│ Приближение │ n_iter Гершорина │ n_iter max наддиагональный │ np_eig - gers_eig   │ np_eig - maxabs_eig  │
├─────────────┼──────────────────┼────────────────────────────┼─────────────────────┼──────────────────────┤
│ 0.01        │ 22               │ 24                         │ 0.03589168301808101 │ 0.03559755016570404  │
├─────────────┼──────────────────┼────────────────────────────┼─────────────────────┼──────────────────────┤
│ 0.001       │ 35               │ 45                         │ 0.0500346820546895  │ 0.04992124045981649  │
├─────────────┼──────────────────┼────────────────────────────┼─────────────────────┼──────────────────────┤
│ 0.0001      │ 35               │ 64                         │ 0.0500346820546895  │ 0.048854632311810184 │
├─────────────┼──────────────────┼────────────────────────────┼─────────────────────┼──────────────────────┤
│ 1e-05       │ 35 